# Nucleotide percentage plots

This notebook contains code to process the "Nucleotide_percentage_table" file from the CRISPResso output into a file containing the % of each each nucleotide at each target nucleotide, numbered relative to the sgRNA. It also produces a plot showing the same information (as in Figure S4A, D, G, J).

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
from pathlib import Path

from IPython.display import Javascript, display

#import aa_guideseq_visualization as guideseq
mpl.rc('pdf', fonttype=42)
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"

In [2]:
from IPython.display import HTML
import random

def hide_toggle(toggle_text_addon = '', for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'
    
    toggle_text = 'Show/Hide' + ' ' + toggle_text_addon  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)
hide_toggle(toggle_text_addon='Toggle Function')

In [3]:
# Check to make sure folder filepaths end in '/' but do not begin with a '/'
def check_folder_filepath(filepath):
    if filepath[0] == '/':
        filepath = filepath[1:]
    if filepath[-1] != '/':
        filepath = filepath+'/'
    return filepath

hide_toggle(toggle_text_addon='check_folder_filepath function')

## User inputs

<font color='blue'> Please follow steps indicated in blue, then run the notebook to generate output files. If the files are formatted as described in the documentation, the code in the 'Functions' section should not need to be altered. </font> 

**Metainformation file** 

<font color='blue'> <b>Step 1:</b> Please enter the filepath to the metainformation input file used in the allele frequencies notebook. The same file will be used in this notebook. </font> 

In [4]:
global input_file
# input_filepath = '../../Metainfo_input_CBE_TP53_first_last_codon.csv'
input_filepath = input("Please enter input filepath here: ")
input_file = pd.read_csv(input_filepath)

input_file

Please enter input filepath here: ../../AudreyData/TP53/Metainfo_input_ABE_TP53_fixed_updated_1d_sample.csv


,sg,sgRNA_sequence,translation_ref_seq,BEV_start,BEV_end,primer,frame,first_codon,last_codon,rev_com,BEV_ref,BEV_test
0,1d,GCTCCTCCATGGCAGTGACC,[TTCCTCTTGCAGCAGCCAGACTGCCTTCCGGGTCACTGCC]ATGG...,417,426,F3_R2,1,ATG,CTG,True,417;418,425;426


<font color='blue'><b>Step 2:</b> Enter filepath to folder containing CRISPResso output files here. Please make sure that the filepath ends in a '/'.  </font> 

Please note that each folder containing CRISPResso output files for individual samples within the given folder should be named in the format 'CRISPResso_on_'+bev+'\_'+
primer, where bev = ('BEV' OR 'NGBEV') + sample_number and primer = primer name. 
Ex. <font color='grey'>CRISPResso_on</font><font color='purple'>_BEV_001</font><font color='green'>_F2_R2</font>

In [5]:
global bev_string_id
bev_string_id = input('Please enter either \'BEV\' or \'NGBEV\' to indicate which string is used when naming your CRISPResso files.')
if ((bev_string_id != 'BEV') and (bev_string_id != 'NGBEV')):
    raise Exception('Invalid input. Please enter either \'BEV\' or \'NGBEV\' to specify which string is used in CRISPResso file names. Be careful not to add any extra spaces.')


Please enter either 'BEV' or 'NGBEV' to indicate which string is used when naming your CRISPResso files.NGBEV


In [6]:
global CRISPResso_filepath
# CRISPResso_filepath= "AudreyData/CRISPRessoBatch_on_F2R1_batch_file_v2/"
CRISPResso_filepath = input("Please enter CRISPResso filepath here: ")
CRISPResso_filepath = check_folder_filepath(CRISPResso_filepath)
print(CRISPResso_filepath)


Please enter CRISPResso filepath here: ../../AudreyData/TP53/TP53_ABE_Sample_CRISPResso/
../../AudreyData/TP53/TP53_ABE_Sample_CRISPResso/


<font color='blue'><b>Step 3:</b> Enter filepath to folder where the files generated by this notebook will be stored. Please make sure that the filepath ends in a '/'. If the folders in this file path do not currently exist, they will be created when the notebook is run.  </font> 

In [7]:
global output_filepath
# output_filepath = 'AudreyData/Validation_CRISPResso_results/'
output_filepath = input("Please enter output folder filepath here: ")
output_filepath = check_folder_filepath(output_filepath)
print(output_filepath)

Please enter output folder filepath here: ../../AudreyData/TP53/ABE/
../../AudreyData/TP53/ABE/


<font color='blue'><b>Step 4:</b> Please select the type of base editor (BE) used in the samples in input file. Then, click on the next cell to continue.

In [8]:
global be_type

be_type_input = input("Please specify the type of base editor used in the input samples by entering 'A' for A base editor or 'C' for C base editor: ")


Please specify the type of base editor used in the input samples by entering 'A' for A base editor or 'C' for C base editor: A


In [9]:
# Make sure a base editor is selected and not default value

if (be_type_input != 'A') and (be_type_input != 'C'):
    raise Exception('Invalid input. Please enter either A or C to specify base editor.')

else:
    # Run rest of notebook! 
    be_type = be_type_input + 'BE'
    display(Javascript('IPython.notebook.execute_cells_below()'))

<IPython.core.display.Javascript object>

<font color='blue'> <b>Ready to run functions!</b> </font> 

## Functions 

In [10]:
'''
This function removes any NaN rows from input_file
'''
def clean_input_file(df):
    df = df.dropna() #drop NaN 
    #dropna() converts int to float, so convert them back
    float_cols = df.select_dtypes(include=['float64']).columns #select subset of df of type float
    for col in float_cols: 
        #get original column index so can replace at correct loc
        index = df.columns.get_loc(col)
        #rename float cols as "float_"col name 
        float_col_name = 'float_' + col
        df = df.rename(columns = {col : float_col_name})
        #overwrite as type int
        float_to_int = df[float_col_name].astype(int).copy()
        df.insert(index, col, float_to_int)
        #drop float column 
        df = df.drop(float_col_name, axis = 1)
    return df

#make input df with columns BEV, offset, rev_com, left_lim, right_lim, primer, width, height
input_df = pd.DataFrame()

#check for NaN values i.e. blank rows
if input_file.isnull().values.any(): 
    input_file = clean_input_file(input_file)

allele_freq_input_df = input_file

# guides = []

BEV_test_df = allele_freq_input_df[['sgRNA_sequence', 'BEV_test', 'primer']].copy()
BEV_test_df['type'] = 'test'
BEV_test_df = BEV_test_df.copy().rename(columns = {'sgRNA_sequence': 'guide_seq', 'BEV_test':'BEV'})

BEV_ref_df = allele_freq_input_df[['sgRNA_sequence', 'BEV_ref', 'primer']].copy()
BEV_ref_df['type'] = 'ref'
BEV_ref_df = BEV_ref_df.copy().rename(columns = {'sgRNA_sequence': 'guide_seq', 'BEV_ref':'BEV'})

input_df = pd.concat([BEV_ref_df, BEV_test_df]).reset_index(drop=True)

# input_df['BEV'] = allele_freq_input_df['BEV_test']

# for i,row1 in allele_freq_input_df.iterrows():
#     guides.append(row1.copy()['sgRNA_sequence'])
    
# input_df['guide_seq'] = guides
# # input_df['rev_com'] = allele_freq_input_df['rev_com'].copy()
input_df['left_lim'] = -25
input_df['right_lim'] = 25
# input_df['primer'] = allele_freq_input_df['primer'].copy()
input_df['width'] = 6 
input_df['height'] = 6 
#input_df = input_df.head(n=2)
input_df

,guide_seq,BEV,primer,type,left_lim,right_lim,width,height
0,GCTCCTCCATGGCAGTGACC,417;418,F3_R2,ref,-25,25,6,6
1,GCTCCTCCATGGCAGTGACC,425;426,F3_R2,test,-25,25,6,6


In [11]:
def get_bev_str(bev):
# Converts BEV number from int to 3-digit string    
    bev = int(bev)
    if bev < 10:
        return '00'+str(bev)
    if bev < 100:
        return '0'+str(bev)
    return str(bev)

def check_filepath(filepath,bev,primer):
    file_loc = filepath+'/CRISPResso_on_NGBEV_'+bev+'_'+primer+'/'+'Nucleotide_percentage_table.txt'
    if path.exists(file_loc):
        return file_loc
    else:
        raise ValueError('File not found. Please check your filepath inputs.')
        return 0
        
def get_complement(val):
    complement_dict = {'A':'T','C':'G','G':'C','T':'A'}
    return complement_dict[val]

def get_bev_df(bev_list,output_name,primer, guide):
    for i,BEV in enumerate(bev_list):
        filepath = CRISPResso_filepath
        file_loc = check_filepath(filepath,get_bev_str(BEV),primer)
        data = pd.read_table(file_loc,header=None)
        data = data.transpose()
        # Set first row to be column names
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.rename(columns = {np.nan:'WT'})
        # Add row showing the position
        data['position'] = data.index - 1
        data = data.rename(columns = {'-':'del'}) # to display better in Excel
        
        amplicon = data['WT'].copy().str.cat()

        offset = amplicon.find(guide)
        print(guide)
        
        # Calculate "offset_position" relative to sgRNA using manually-determined offset (custom for each sgRNA)
        data['offset_position'] = data['position'] - offset + 1
#         print(data.loc[guide_start_index:guide_end_index, :])
        
        data = data.rename(columns = {'A':'A_'+str(BEV),'C':'C_'+str(BEV),'G':'G_'+str(BEV),'T':'T_'+str(BEV),'del':'del_'+str(BEV),'N':'N_'+str(BEV)})

        if i == 0:
            existing_df = data
        else:
            # Merge onto existing dataframe
            existing_df = pd.merge(existing_df,data,on=['WT','position','offset_position'],how='outer')
    
    # Average columns
    for nuc in ['A','C','G','T','N','del']:
        cols = []
        for bev in bev_list:
            cols.append(nuc+'_'+bev)
        existing_df[cols] = existing_df[cols].astype(float)
        existing_df[nuc+'_avg'] = existing_df[cols].mean(axis=1)
    
    # Write out file
    Path(output_filepath + "nucleotide_percentage/").mkdir(parents=True, exist_ok=True)
    existing_df.to_csv(output_filepath + 'nucleotide_percentage/BEV_'+output_name+'.csv',index=False)
    
    # Filter for rows where WT is A or C depending on base editor type 
    if be_type == 'ABE':
        existing_df = existing_df.loc[existing_df['WT'] == 'A']
    elif be_type == 'CBE':
        existing_df = existing_df.loc[existing_df['WT'] == 'C']

    return existing_df


In [12]:
def make_plot_v2(data,left_lim,right_lim,bevs,primer,width,height):
    
    # Get list of value_vars for pd.melt
    value_vars = []
    if be_type == 'ABE':
        nuc_list = ['C','T','G','N','del']
    elif be_type == 'CBE':
        nuc_list = ['A','T','G','N','del']
    
    for nuc in nuc_list:
        for bev in bevs:
            value_vars.append(nuc+'_'+str(bev))
        value_vars.append(nuc+'_avg')
    
    # Make tidy data
    data = data.melt(id_vars=['WT','offset_position'],value_vars=value_vars,
                    var_name='nucleotide',value_name='percentage')
    
    # Filter for nucleotides to include on plot
    data = data[(data['offset_position'] > left_lim) & (data['offset_position'] < right_lim)]
    data = data.sort_values(by='offset_position')
    data['offset_position'] = data['offset_position'].astype(str)
    
    # Convert to percentage
    data['percentage'] = data['percentage'] * 100
    
    # Split "nucleotide" column into two: one with BEV number (or average) and one with nucleotide
    data['BEV'] = data['nucleotide'].apply(lambda x: x.split('_')[1]) # returns BEV number or "avg"
    data['nucleotide'] = data['nucleotide'].apply(lambda x: x.split('_')[0]) # returns nucleotide
    
    # Get order of offset positions for plotting
    if be_type == 'ABE': 
        order = data.loc[(data['nucleotide'] == 'C') & (data['BEV'] == 'avg'),'offset_position'].tolist() # require that 'BEV' == 'avg' just to deduplicate list
        hue_order = ['C','G','T','N','del']
    
    elif be_type == 'CBE': 
        order = data.loc[(data['nucleotide'] == 'A') & (data['BEV'] == 'avg'),'offset_position'].tolist() # require that 'BEV' == 'avg' just to deduplicate list
        hue_order = ['A','G','T','N','del']
    
    # Make plot
    fig,ax = plt.subplots(figsize=(width,height))
    sns.set_context(rc = {'patch.linewidth': 0.0})
    
    # Plot average of 2 replicates as bar    
    sns.barplot(x='offset_position',y='percentage',hue='nucleotide',data=data.loc[data['BEV'] == 'avg',:],order=order,hue_order=hue_order,palette=sns.color_palette('Set2'),
               linewidth=0)
    
    # Plot individual replicates as dots
    for bev in bevs:
        sns.stripplot(x='offset_position',y='percentage',hue='nucleotide',data=data.loc[data['BEV'] == bev,:],order=order,hue_order=hue_order,color='black',s=1,dodge=True)
    if be_type == 'ABE':
        plt.xlabel('position of A',fontsize=6)
    elif be_type == 'CBE':
        plt.xlabel('position of C',fontsize=6)
    plt.ylabel('percentage',fontsize=6)
    
    # Set y axis to start at 0
    ylim_upper = ax.get_ylim()[1]
    ax.set_ylim(0,ylim_upper)
    
    # Clean up axes
    sns.despine()
    for axis in ['bottom','left']:
        ax.spines[axis].set_linewidth(0.5)
    ax.tick_params(axis='both',labelsize=6,width=0.5,length=2)
    
    # Plot legend 
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles[-5:], labels[-5:],loc='upper right',frameon=False,fontsize=6)
    Path(output_filepath + "nucleotide_percentage/").mkdir(parents=True, exist_ok=True)
    fig.savefig(output_filepath + 'nucleotide_percentage/BEV_'+output_name+'.pdf',transparent=True,bbox_inches = "tight")
    plt.close()
    
    return

In [13]:
# Get input data

for i,row in input_df.iterrows():
    print(row['BEV'])
    bev_list = row['BEV'].split(';')
    output_name = '_'.join(bev_list)+'_'+row['primer']
    #bev_df = get_bev_df(bev_list,row['rev_com'],output_name,row['primer'], row['guide_seq'])
    bev_df = get_bev_df(bev_list,output_name,row['primer'],row['guide_seq'])
    make_plot_v2(bev_df,row['left_lim'],row['right_lim'],bev_list,output_name,row['width'],row['height'])
    
    #break

417;418
GCTCCTCCATGGCAGTGACC
GCTCCTCCATGGCAGTGACC
425;426
GCTCCTCCATGGCAGTGACC
GCTCCTCCATGGCAGTGACC
